## 1D Example

$$-\partial_x ( k(x) \partial_x u) = f,$$


#### Higher dimensions
The same approach works in higher dimensions, the derivative just need to be taken in each direction. In the coursework example this means that most derivatives for the first summand will drop away.

Let's start with the equation for $k(x)=1$

In this case:

$$f(x) = \pi^2 \sin(\pi x)$$

and the exact solution is:

$$u^*(x) = \sin(\pi x)$$

In [ ]:
%matplotlib notebook
from matplotlib import pyplot
import numpy
pyplot.style.use('ggplot')

N = 100
h = 1/(N-1)
pi = numpy.pi

In [ ]:
def laplacian(N, rhsfunc):
    x = numpy.linspace(0, 1, N+1)
    h = 1/N
    rhs = rhsfunc(x)
    e = numpy.ones(N)
    # interior discretisation
    L = (2*numpy.eye(N+1) - numpy.diag(e, 1) - numpy.diag(e, -1)) / h**2
    return x, L, rhs, h

In [ ]:
def apply_dirichlet(L, rhs, h, vals, indices):
    N, _ = L.shape
    diag = numpy.eye(1, N)
    bcmask = numpy.zeros(N, dtype=bool)
    bcmask[indices] = True
    # Dirichlet rows
    L[numpy.ix_(bcmask)] = numpy.vstack([numpy.roll(diag, i) for i in indices])
    rhs[numpy.ix_(bcmask)] = vals
    # Forward substitute
    rhs[numpy.ix_(~bcmask)] -= L[numpy.ix_(~bcmask, bcmask)] @ vals
    L[numpy.ix_(~bcmask, bcmask)] = 0
    return L, rhs, h

In [ ]:
# rhs
def f(x):
    return pi**2 * numpy.sin(pi*x)

# exact
def ex(x):
    return numpy.sin(pi*x)

x, L, rhs, h = laplacian(N, f)
L, rhs, h = apply_dirichlet(L, rhs, h, [0,0], [0,N])
u = numpy.linalg.solve(L, rhs)

In [ ]:
x = numpy.linspace(0,1,N+1)

pyplot.figure()
pyplot.plot(x,u)
pyplot.plot(x,ex(x),'x')
pyplot.show()

In [ ]:
# Here we use the 2-norm
def error(u, exact, h):
    return numpy.sqrt(h)*numpy.linalg.norm(u - exact)

def mms_errors(exact,rhsfunc):
    errors = []
    Ns = numpy.asarray(list(2**i for i in range(4, 11)))
    for N in Ns:
        x, L, rhs, h = laplacian(N, rhsfunc)
        L, rhs, h = apply_dirichlet(L, rhs, h, [0,0], [0,N])
        u = numpy.linalg.solve(L, rhs)
        errors.append(error(u, exact(x), 1/N))
    return 1/Ns, numpy.asarray(errors)

hs, errors = mms_errors(ex,f)
pyplot.figure()
pyplot.loglog(hs, errors, "o", label="Error");
pyplot.loglog(hs, hs, label="$\mathcal{O}(h)$");
pyplot.loglog(hs, hs**2, label="$\mathcal{O}(h^2)$");
pyplot.legend();
pyplot.show()

And now let's try the same thing for the variable coefficient case:

Let 
$$k(x) = \cos(\pi x)$$

The exact solution remains the same and (in the 3D case) we adjust the right hand side. Discretise by using product rule:

$$\partial_x ( k(x) \partial_x u) = \partial_x k(x) \partial_x u + k \partial_x^2 u$$

So now in addition to the stencil for the laplacian we already have from above we need the stencil for the single derivative and evaluations of $k$ and its derivative at the nodal points.

In [ ]:
def operator(N, rhsfunc):
    x = numpy.linspace(0, 1, N+1)
    h = 1/N
    rhs = rhsfunc(x)
    e = numpy.ones(N)
    # interior discretisation consists of two components
    # k * laplacian component
    L = numpy.cos(pi * x * h) * (2*numpy.eye(N+1) - numpy.diag(e, 1) - numpy.diag(e, -1)) / h**2
    # k' * centered derivative
    L+= numpy.sin(pi * x * h) * (numpy.diag(e,-1) - numpy.diag(e,1)) / h    
    return x, L, rhs, h

In [ ]:
# rhs
def f(x):
    return pi**2 *  numpy.sin(pi*x)

# exact
def ex(x):
    return numpy.sin(pi*x)

x, L, rhs, h = operator(N, f)
L, rhs, h = apply_dirichlet(L, rhs, h, [0,0], [0,N])
unum = numpy.linalg.solve(L, rhs)

In [ ]:
x = numpy.linspace(0,1,N+1)
pyplot.figure()
pyplot.plot(x,unum)
pyplot.plot(x,ex(x),'x')
pyplot.show()

In [ ]:
# Here we use the 2-norm
def error(u, exact, h):
    return numpy.sqrt(h)*numpy.linalg.norm(u - exact)

def mms_errors(exact,rhsfunc):
    errors = []
    Ns = numpy.asarray(list(2**i for i in range(4, 11)))
    for N in Ns:
        x, L, rhs, h = laplacian(N, rhsfunc)
        L, rhs, h = apply_dirichlet(L, rhs, h, [0,0], [0,N])
        u = numpy.linalg.solve(L, rhs)
        errors.append(error(u, exact(x), 1/N))
    return 1/Ns, numpy.asarray(errors)

hs, errors = mms_errors(ex,f)
pyplot.figure()
pyplot.loglog(hs, errors, "o", label="Error");
pyplot.loglog(hs, hs, label="$\mathcal{O}(h)$");
pyplot.loglog(hs, hs**2, label="$\mathcal{O}(h^2)$");
pyplot.legend();

Similarly, instead of the direct solve we could solve by using an explicit Euler method

In [ ]:
def explicit_euler(u0, L, dt, T=500):
    us = [u0]
    ts = [0]
    update = numpy.zeros_like(u0)
    import copy
    u = copy.copy(u0)
    t = 0 
    while t < T:
        update = rhs - L @ u
        if numpy.linalg.norm(update, numpy.inf) < 1e-8:
            print("Equilibrium reached at t=",t)
            # Terminate if we've reached a steady-state
            break
        # Explicit Euler: u <- u + dt f(u)
        u += dt*update
        us.append(u)
        t += dt
        ts.append(t)
    return ts, us

u0 = numpy.zeros(N+1)
# You'll need a very small time step for this one
ts, us = explicit_euler(u0, L, 0.00001, T=3.0)

In [ ]:
x = numpy.linspace(0,1,N+1)
pyplot.figure()
pyplot.plot(x,us[-1])
pyplot.plot(x,ex(x),'x')
pyplot.show()